In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

data = pd.read_csv('data/german_credit.csv')
data.head()

C:\Users\leiyo\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\leiyo\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

target_name = 'Risk'

# Encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Split the data into features and target variable
X = data.drop(target_name, axis=1)
y = data[target_name]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy = {accuracy}')
print(report)


Accuracy = 0.755
              precision    recall  f1-score   support

           0       0.65      0.37      0.47        59
           1       0.78      0.91      0.84       141

    accuracy                           0.76       200
   macro avg       0.71      0.64      0.66       200
weighted avg       0.74      0.76      0.73       200



In [5]:
import numpy as np

sensitive_attribute = 'Sex'

# Extract the coefficient for the "sex" feature
sex_coefficient = model.coef_[0][X.columns.get_loc(sensitive_attribute)]

# Check the distribution of predictions for different sexes
predicted_proba = model.predict_proba(X_test)[:, 1]
X_test['predicted_proba'] = predicted_proba
X_test['actual'] = y_test

# Group by sex
grouped_by_sex = X_test.groupby(sensitive_attribute).agg({
    'predicted_proba': ['mean'],
    'actual': ['mean']
})

sex_coefficient, grouped_by_sex


(0.4317750957821051,
     predicted_proba    actual
                mean      mean
 Sex                          
 0          0.670443  0.696429
 1          0.715529  0.708333)

In [40]:
from src.matching import NearestNeighborDataMatcher

In [41]:
X_labeled, X_unlabeled = X_train.head(200), X_train.tail(600)

In [42]:
matching = NearestNeighborDataMatcher(X_labeled=X_labeled, X_unlabeled=X_unlabeled).match(n_neighbors=2)